In [4]:

# ## 1) Install dependencies
# !pip install transformers torch torchvision pillow scikit-learn --quiet

# ## 2) Imports & setup
import os
import time
import pandas as pd
import torch
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoConfig,
    AutoImageProcessor,
    AutoModelForImageClassification
)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Paths
CSV_PATH  = "challenge-2016-test_metadata_2025-05-14.csv"
IMAGE_DIR = "ISIC-images"


NameError: name '_C' is not defined

In [ ]:

# %% [markdown]
# ## 3) Load & prepare DataFrame
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["benign_malignant"])
df["label"] = df["benign_malignant"].str.lower().map({"benign": 0, "malignant": 1})
df = df[df["label"].isin([0,1])].reset_index(drop=True)

train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=42
)
print(f"Train/Val sizes: {len(train_df)} / {len(val_df)}")

In [ ]:

# %% [markdown]
# ## 4) Processor & Dataset
model_name = "Anwarkh1/Skin_Cancer-Image_Classification"
processor  = AutoImageProcessor.from_pretrained(model_name)

class ISICDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(IMAGE_DIR, f"{row['isic_id']}.jpg")
        img = Image.open(img_path).convert("RGB")
        proc = processor(images=img, return_tensors="pt")
        pixel_values = proc["pixel_values"].squeeze(0)
        label = torch.tensor(row["label"], dtype=torch.long)
        return pixel_values, label

train_ds = ISICDataset(train_df)
val_ds   = ISICDataset(val_df)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=16)


In [ ]:

# %% [markdown]
# ## 5) Load model with fresh binary head
#    (drops the original 7-way head and initializes a new 2-way head)
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0:"benign", 1:"malignant"},
    label2id={"benign":0, "malignant":1},
)
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True
).to(device)


In [ ]:
# %% [markdown]
# ## 6) Optimizer, scheduler, loss
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
total_steps = len(train_loader) * 5  # if you run 5 epochs
scheduler   = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)
criterion   = torch.nn.CrossEntropyLoss()

In [ ]:
# %% [markdown]
# ## 7) Evaluation helper
def evaluate():
    model.eval()
    preds, labs = [], []
    with torch.no_grad():
        for px, lb in val_loader:
            px, lb = px.to(device), lb.to(device)
            out = model(pixel_values=px).logits
            preds.extend(out.argmax(-1).cpu().numpy())
            labs.extend(lb.cpu().numpy())
    return {
        "accuracy":  accuracy_score(labs, preds),
        "precision": precision_score(labs, preds),
        "recall":    recall_score(labs, preds),
        "f1":        f1_score(labs, preds)
    }


In [ ]:

# %% [markdown]
# ## 8) Training loop
num_epochs = 5
best_f1    = 0.0
start_time = time.time()

for epoch in range(1, num_epochs+1):
    model.train()
    for px, lb in train_loader:
        px, lb = px.to(device), lb.to(device)
        optimizer.zero_grad()
        logits = model(pixel_values=px).logits
        loss   = criterion(logits, lb)
        loss.backward()
        optimizer.step()
        scheduler.step()

    metrics = evaluate()
    print(f"Epoch {epoch} — "
          f"Acc: {metrics['accuracy']:.3f}, "
          f"Prec: {metrics['precision']:.3f}, "
          f"Rec: {metrics['recall']:.3f}, "
          f"F1: {metrics['f1']:.3f}")

    if metrics["f1"] > best_f1:
        best_f1 = metrics["f1"]
        torch.save(model.state_dict(), "best_head.pt")

elapsed = time.time() - start_time
print(f"Done in {elapsed/60:.1f} min — Best F1: {best_f1:.3f}")


Device: cpu
Train/Val sizes: 300 / 76


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at Anwarkh1/Skin_Cancer-Image_Classification and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\benbu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this b

Epoch 1 — Acc: 0.803, Prec: 0.000, Rec: 0.000, F1: 0.000
Epoch 2 — Acc: 0.842, Prec: 1.000, Rec: 0.200, F1: 0.333
Epoch 3 — Acc: 0.842, Prec: 0.714, Rec: 0.333, F1: 0.455
Epoch 4 — Acc: 0.842, Prec: 0.714, Rec: 0.333, F1: 0.455
Epoch 5 — Acc: 0.842, Prec: 0.714, Rec: 0.333, F1: 0.455
Done in 52.4 min — Best F1: 0.455
